In [1]:
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)



from tensorflow import keras
import time

from functions_dataCreation import *
from functions_modelArchitectures import *
from class_LRFinder import *


from tensorflow.keras.callbacks import Callback

import pandas as pd

import wandb
from wandb.keras import WandbCallback


set_x,set_y = readXYfromDisk(noOfFiles=20,folder="../data/Train") #Training data
test_set_x,test_set_y = readXYfromDisk(noOfFiles=2,folder='../data/Test') 

set_y[set_y == 1] = 0
set_y[set_y == 2] = 1

test_set_y[test_set_y == 1] = 0
test_set_y[test_set_y == 2] = 1

IMG_SIZE = 128

METRICS = [
    #   keras.metrics.TruePositives(name='tp'),
    #   keras.metrics.FalsePositives(name='fp'),
    #   keras.metrics.TrueNegatives(name='tn'),
    #   keras.metrics.FalseNegatives(name='fn'), 
    #   keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
]


1 Physical GPUs, 1 Logical GPUs
X Shape :  (85064, 128, 128, 3) Memory size is : 3987.375 Mb Y Shape:  (85064,)
Values, counts, Avg Performance :  [0 1 2] [44005 20023 21036] [0.51731637 0.23538747 0.24729615]
X Shape :  (10060, 128, 128, 3) Memory size is : 471.5625 Mb Y Shape:  (10060,)
Values, counts, Avg Performance :  [0 1 2] [4988 2453 2619] [0.49582505 0.24383698 0.26033797]


# Train longer
 
* Train a CNN for 10000 epochs with LR of 1e-5
* PC crashed at 6000 epochs, took around 1.5 days
* Results are on [wandb](https://wandb.ai/amitagni/candlestick-CNN/runs/27gxv7mr?workspace=user-amitagni)


In [ ]:
wandb.init(project="candlestick-CNN", name = 'Deeper 1e-5 10K epochs' )


modelBase = {}
modelBase['name'] = 'Base'
modelBase['inputShape'] = (IMG_SIZE,IMG_SIZE,3)
modelBase['activation'] = 'relu'


modelBase['convLayerMultiplier'] = 1

modelBase['poolingLayer'] = 'MaxPooling2D'
modelBase['padding'] = 'same'

modelBase['denseLayers'] = 2
modelBase['units'] = 128
modelBase['activation'] = 'relu'

#with Dropout and BN
modelBase_with_Dropout = modelBase.copy()
modelBase_with_Dropout['name'] = 'modelBase_with_Dropout'
modelBase_with_Dropout['batchnormalization'] = False
modelBase_with_Dropout['dropout'] = 0.01

modelBase_with_Dropout['kernelSize'] = (3,3)
modelBase_with_Dropout['filters'] = [10,15,20,25,30,35,40,45,50,55,60]
modelBase_with_Dropout['poolSize'] = (6,6)

df = pd.DataFrame()

for imgs in [85000]:
    for lr in [1e-5]:

        start_time = time.time()

        set_x = set_x[:imgs]
        set_y = set_y[:imgs]
        
        model = createCNN(modelBase_with_Dropout)
        
        model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr)
                ,loss=tf.keras.losses.binary_crossentropy
                ,metrics=[METRICS])

        history = model.fit(set_x,set_y
                        ,batch_size = 128
                        ,epochs=10000
                        ,verbose=0   
                        ,validation_data=(test_set_x,test_set_y)
                        ,validation_freq = 50
                        ,callbacks=[WandbCallback()]                                        
                        )

        temp = pd.DataFrame(history.history).rename_axis("epoch")
        
        temp['elapsed'] = round((time.time() - start_time)/60,2)

        var_params = "Deeper_" + "imgs_" + str(imgs) + "_LR_"  + str(lr)

        temp['params'] = var_params

        print("Elapsed time " + str(round((time.time() - start_time)/60,2)) + var_params)


## Faster LR

Changed glorot_normal for relu activation

In [2]:

wandb.init(project="candlestick-CNN", name = 'Deeper 1e-4 10K epochs' )

modelBase = {}
modelBase['name'] = 'Base'
modelBase['inputShape'] = (IMG_SIZE,IMG_SIZE,3)
modelBase['activation'] = 'relu'

modelBase['convLayerMultiplier'] = 1

modelBase['poolingLayer'] = 'MaxPooling2D'
modelBase['padding'] = 'same'

modelBase['denseLayers'] = 2
modelBase['units'] = 128
modelBase['activation'] = 'relu'

#with Dropout and BN
modelBase_with_Dropout = modelBase.copy()
modelBase_with_Dropout['name'] = 'modelBase_with_Dropout'
modelBase_with_Dropout['batchnormalization'] = False
modelBase_with_Dropout['dropout'] = 0.05

modelBase_with_Dropout['kernelSize'] = (3,3)
modelBase_with_Dropout['filters'] = [10,15,20,25,30,35,40,45,50,55,60]
modelBase_with_Dropout['poolSize'] = (6,6)

lr = 1e-4
imgs = 80000

df = pd.DataFrame()

start_time = time.time()

set_x = set_x[:imgs]
set_y = set_y[:imgs]

checkpoint_path = '../data/callbacks/'

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=500)
    
  

model = createCNN(modelBase_with_Dropout)

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=lr)
        ,loss=tf.keras.losses.binary_crossentropy
        ,metrics=[METRICS])

history = model.fit(set_x,set_y
                ,batch_size = 128
                ,epochs=10000
                ,verbose=0   
                ,validation_data=(test_set_x,test_set_y)
                ,validation_freq = 50
                ,callbacks=[WandbCallback()
                            ,cp_callback]                                        
                )

temp = pd.DataFrame(history.history).rename_axis("epoch")
temp['elapsed'] = round((time.time() - start_time)/60,2)
var_params = "Deeper_" + "imgs_" + str(imgs) + "_LR_"  + str(lr)
temp['params'] = var_params

print("Elapsed time " + str(round((time.time() - start_time)/60,2)) + var_params)

model.save('../data/savedmodels') 


Failed to query for notebook name, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable
wandb: Currently logged in as: amitagni (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.9 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


KeyboardInterrupt: 